In [6]:
import pandas as pd
import numpy as np
import pickle
import math
from heapq import nlargest

In [7]:
file_to_read = open("user2movie.json", "rb")
user2movies = pickle.load(file_to_read)

file_to_read = open("movie2user.json", "rb")
movie2users = pickle.load(file_to_read)

file_to_read = open("usermovie2rating.json", "rb")
usermovie2rating = pickle.load(file_to_read)

file_to_read = open("usermovie2rating_test.json", "rb")
usermovie2rating_test = pickle.load(file_to_read)

In [8]:
def movie_avg_rating(movie_id):
    #getting movie id 
    #return its avg rating over all users who rated it
    count_rating =0
    sum_rating = 0
    for user in movie2users[movie_id]:#iterating over all user id rated movies
        sum_rating += usermovie2rating[(user, movie_id)]#sum the rating
        count_rating+=1
    avg_rating = sum_rating/count_rating
    return avg_rating

In [9]:
def calc_single_w(movie_a, movie_b, movie_avg):
#getting two user's id, returning the weight by peerson
    movie_a_mean = movie_avg[movie_a]#R_a- need to create this list
    movie_b_mean = movie_avg[movie_b]#R_b
    numerator = 0
    denominator = 0
    dom_a = 0
    dom_b = 0
    #finding the subset of intersection between users who rated both a and b
    set_a = movie2users[movie_a]
    set_b = movie2users[movie_b]
    intersection = set(set(set_a) & set(set_b))
    if len(intersection)<5:#if intersection to low, ignore this item
        return 0
    for user in intersection:
        movie_rating_a = usermovie2rating[(user, movie_a)]
        movie_rating_b = usermovie2rating[(user, movie_b)]
        movie_dev_a = movie_rating_a - movie_a_mean#user a rating deviation of movie
        movie_dev_b = movie_rating_b - movie_b_mean#user b rating deviation of movie
        numerator += movie_dev_a*movie_dev_b
        dom_a += movie_dev_a**2
        dom_b += movie_dev_b**2
    dom_a = math.sqrt(dom_a)
    dom_b = math.sqrt(dom_b)
    denominator = dom_a*dom_b
    w = numerator/denominator
    return w

In [10]:

def calculate_all_w(movie_avg):
    #calculating all the weights, putting into dict
    movie_movie_w = {}

    for movie_i in movie2users.keys():#running over all users
        for movie_j in movie2users.keys():
            if movie_i == movie_j:
                continue
            if (movie_j, movie_i) in movie_movie_w:
                movie_movie_w[(movie_i, movie_j)] = movie_movie_w[(movie_j, movie_i)]#correlation is symetric so if already in dict no need to calc again
            else:
                 movie_movie_w[(movie_i, movie_j)] = calc_single_w(movie_i, movie_j, movie_avg)
        
    return movie_movie_w

In [11]:

def neighborfy(movie_id, movie_movie_w, relevant_items):
#should get a subset of movies top correlated to movie_id
   
    k=30
    movieid_movie_w = {}
    for movie in relevant_items:#looping throw subset of movies rated by userid
        if movie == movie_id:
            continue
        else:
            movieid_movie_w[movie] = abs(movie_movie_w[movie_id, movie])#taking the two movies weight to the new movie_id dict
    
    neighbors = nlargest(k, movieid_movie_w, key = movieid_movie_w.get)#getting the k best neighbors
    return neighbors

In [12]:
def get_score(movie_id, user_id, movie_movie_w, movie_avg):

    items_rated_by_user_full= user2movies[user_id]# subset of movies rated by userid
    items_rated_by_user = neighborfy(movie_id, movie_movie_w, items_rated_by_user_full) #the top 30

    w = {}# a dict containing all the weights between movie_id and other movies from his neighbors 

    movieId_mean = movie_avg[movie_id]#user_id avg rating of all movies
    #TO DO- calc in advance the rating- not depended on movie!
    numerator = 0
    denominator = 0
    for movie in items_rated_by_user:
        w[movie]= movie_movie_w[(movie_id, movie)]# fetch current user and user_id weight. 
        user_rating= movie_avg[movie]#get current user avg
        deviation = user_rating - movieId_mean
        numerator+= w[movie]*deviation
        denominator = denominator+abs(w[movie])
    
    
    if denominator == 0:
        return -10000  
    score = movieId_mean+ (numerator/denominator)
    return score

In [13]:
def calc_mse(test, pred):
    error =0
    count = 0
    for pair in test.keys():
        error += (test[pair]-pred[pair])**2
        count+=1
    mse = error/count
    return mse

In [14]:

movie_avg = {}
for movie in movie2users.keys():
    movie_avg[movie] = movie_avg_rating(movie)

In [16]:
w = calculate_all_w(movie_avg)#calculate w on train dicts
with open('weights_item.json', 'wb') as f:
  pickle.dump(w, f)

In [18]:
scores = {}
i=0
errors = []
for pair in usermovie2rating_test.keys():
    user = pair[0]   
    scores[pair] = get_score(pair[1], pair[0], w, movie_avg)#for every pair in the test lets find the score
    if scores[pair] == -10000:
        scores.pop(pair)
        errors.append(pair)
        print("error calculating %s prediction" %(pair,))
    i+=1

In [19]:
print(len(errors))

0


In [20]:
for error in errors:
    usermovie2rating_test.pop(error)
MSE = calc_mse(scores, usermovie2rating_test)
print("mse:", MSE)

mse: 0.8555111951446921
